In [26]:

from __future__ import print_function
import numpy as np
from keras.layers import Input, Dense
import chainer
from chainer import Function, gradient_check, report, training, utils, Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Input, Flatten, LSTM
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from chainer.training import extensions
import os, sys, re
from nltk import tokenize
import nltk
nltk.download('punkt')
import string 

import pandas as pd

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sivagurukannan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [310]:
import sys
print(sys.version)

3.5.2 | packaged by conda-forge | (default, Jul 26 2016, 01:37:38) 
[GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.54)]


In [9]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [10]:
TEXT_DATA_DIR = './speech_data/data/'

In [11]:
# Load the labels 
df = pd.read_csv('./speech_data/presidents_meta.csv')
label_dict = dict(zip(list(df.foldername), list(df.label)))

In [243]:
label_dict['arthur']

4

In [12]:
texts = []  # list of text samples
# labels_index = {}  # dictionary mapping label name to numeric id
labels = []  # list of label ids


for name in sorted(os.listdir(TEXT_DATA_DIR)):

    path = os.path.join(TEXT_DATA_DIR, name)
    if os.path.isdir(path):
        for fname in sorted(os.listdir(path)):
            if 'combined' not in fname and 'combines' not in fname:
                president_name = path.split('/')[-1]
                fpath = os.path.join(path, fname)
                f = open(fpath)
                if (president_name in label_dict):
                    labels.append(label_dict[president_name])
                    texts.append(f.read())
                f.close()
                

print('Found %s texts.' % len(texts))
print('Found %s texts.' % len(labels))

Found 862 texts.
Found 862 texts.


In [13]:
texts[500][:1000]

'THE PRESIDENT. Good afternoon, ladies and gentlemen.I have been asked to give a statement about the consular convention that is pending before the United States Senate.I should like to say very briefly that I hope the Senate will give its advice and consent to the proposed convention with the U.S.S.R. I feel very strongly that the ratification of this treaty is very much in our national interest. I feel this way for two principal reasons:First, we need this treaty to protect 18,000 American citizens who each year travel from this country to the Soviet The convention requires immediate notification to us whenever an American is arrested in the Soviet Union. It insures our right to visit that citizen within 4 and as often thereafter as is desirable.We think that we need these rights help to protect American citizens. These are rights which the Soviet citizens already have who travel in this country, because guaranteed by our Constitution.Second, the convention does not require the openi

In [84]:

    
def find_max_nb_words(texts):
    MAX_NB_WORDS = 0
    for text in texts:
        sents = tokenize.sent_tokenize(text)
        print(len(sents))
        local_max = max(list(map(len, sents)))
        if   local_max > MAX_NB_WORDS  :
            MAX_NB_WORDS =  local_max
        print("Maximum words {}".format(MAX_NB_WORDS))
    return MAX_NB_WORDS
MAX_NB_WORDS = find_max_nb_words(texts)

37
Maximum words 4345
74
Maximum words 4345
55
Maximum words 4345
46
Maximum words 4345
24
Maximum words 4345
38
Maximum words 4345
6
Maximum words 4345
59
Maximum words 4345
3
Maximum words 4345
14
Maximum words 4345
57
Maximum words 4345
40
Maximum words 4345
6
Maximum words 4345
398
Maximum words 4345
148
Maximum words 4345
135
Maximum words 5273
150
Maximum words 5273
24
Maximum words 5273
38
Maximum words 5273
97
Maximum words 5273
13
Maximum words 5273
10
Maximum words 5273
11
Maximum words 5273
21
Maximum words 5273
7
Maximum words 5273
153
Maximum words 5273
8
Maximum words 5273
8
Maximum words 5273
139
Maximum words 5273
299
Maximum words 5273
268
Maximum words 5273
343
Maximum words 5273
322
Maximum words 5273
253
Maximum words 5273
10
Maximum words 5273
18
Maximum words 5273
114
Maximum words 5273
21
Maximum words 5273
37
Maximum words 5273
72
Maximum words 5273
14
Maximum words 5273
205
Maximum words 5273
73
Maximum words 5273
34
Maximum words 5273
81
Maximum words 5273
310

ValueError: max() arg is an empty sequence

In [14]:
def normalize_corpus(text, replace_period=False):
    import string 
    sents = tokenize.sent_tokenize(text)
    sents = map(lambda x: x.replace('.',' '), sents)
    local_max = max(map(len, sents))
    if   local_max > MAX_NB_WORDS:
        MAX_NB_WORDS =  max(map(len, sents))
    print("Maximum words {}".format(MAX_NB_WORDS))
    text = ' . '.join(sents)      
    for char in string.punctuation:
        if not replace_period and char=='.':
            text = text.replace(char, ' . ') #text.replace(char, ' ' + char + ' ')
        else:
            text = text.replace(char, ' ') #text.replace(char, ' ' + char + ' ')
    text = re.sub(r'[^\x00-\x7F]+',' ', text) #remove non-ASCII chars
    text = re.sub( '\s+', ' ', text).lstrip().rstrip() #remove extra and trailing spaces
    return text.lower()
norm_texts = list(map(normalize_corpus, texts, MAX_NB_WORDS))

NameError: name 'MAX_NB_WORDS' is not defined

In [19]:
norm_texts[1]

'the personal inconveniences to the members of the senate and of the house of representatives in leaving their families and private affairs at this season of the year are so obvious that i the more regret the extraordinary occasion which has rendered the convention of congress indispensable it would have afforded me the highest satisfaction to have been able to congratulate you on a restoration of peace to the nations of europe whose animosities have endangered our tranquillity but we have still abundant cause of gratitude to the supreme dispenser of national blessings for general health and promising seasons for domestic and social happiness for the rapid progress and ample acquisitions of industry through extensive territories for civil political and religious liberty while other states are desolated with foreign war or convulsed with intestine divisions the united states present the pleasing prospect of a nation governed by mild and equal laws generally satisfied with the possession

In [20]:
texts[500]

'Mr. Speaker, Mr. Vice President, my copartners in Government, gentlemen and ladies:   The Constitution imposes upon me the obligation to "from time to time give to the Congress information of the State of the Union." While this has traditionally been interpreted as an annual affair, this tradition has been broken in extraordinary times.   These are extraordinary times. And we face an extraordinary challenge. Our strength as well as our convictions have imposed upon this nation the role of leader in freedom\'s cause.   No role in history could be more difficult or more important. We stand for freedom. That is our conviction for ourselves—that is our only commitment to others. No friend, no neutral and no adversary should think otherwise. We are not against any man—or any nation—or any system—except as it is hostile to freedom. Nor am I here to present a new military doctrine, bearing any one name or aimed at any one area. I am here to promote the freedom doctrine.   The great battlegro

In [22]:
' '.join(tokenize.sent_tokenize(texts[500]))

'Mr. Speaker, Mr. Vice President, my copartners in Government, gentlemen and ladies:   The Constitution imposes upon me the obligation to "from time to time give to the Congress information of the State of the Union." While this has traditionally been interpreted as an annual affair, this tradition has been broken in extraordinary times. These are extraordinary times. And we face an extraordinary challenge. Our strength as well as our convictions have imposed upon this nation the role of leader in freedom\'s cause. No role in history could be more difficult or more important. We stand for freedom. That is our conviction for ourselves—that is our only commitment to others. No friend, no neutral and no adversary should think otherwise. We are not against any man—or any nation—or any system—except as it is hostile to freedom. Nor am I here to present a new military doctrine, bearing any one name or aimed at any one area. I am here to promote the freedom doctrine. The great battleground fo

In [23]:
normalize_corpus(texts[500])

'mr speaker mr vice president my copartners in government gentlemen and ladies the constitution imposes upon me the obligation to from time to time give to the congress information of the state of the union while this has traditionally been interpreted as an annual affair this tradition has been broken in extraordinary times these are extraordinary times and we face an extraordinary challenge our strength as well as our convictions have imposed upon this nation the role of leader in freedom s cause no role in history could be more difficult or more important we stand for freedom that is our conviction for ourselves that is our only commitment to others no friend no neutral and no adversary should think otherwise we are not against any man or any nation or any system except as it is hostile to freedom nor am i here to present a new military doctrine bearing any one name or aimed at any one area i am here to promote the freedom doctrine the great battleground for the defense and expansio

In [15]:
MAX_NB_WORDS = 100
len(texts)
mini_test = texts[:10]
print(list(map(len, mini_test)))
MAX_SEQUENCE_LENGTH = 100
MAX_SENTENCES = 100
import numpy as np

[2665, 11080, 10013, 1927, 81934, 62451, 51770, 54751, 5014, 6845]


In [16]:
def pad_dense_list(speech, maxsentence):
    
    each_sub_len = len(speech[0])
    Z = np.zeros((maxsentence, each_sub_len))
    #print("Shape of Z{}, list {}", Z.shape, len(speech))
    for index, row in enumerate(speech):
        if (index >= maxsentence):
            break
        
        Z[index] = row
    #print(Z) 
    return Z

In [17]:
def preprocess_text(mini_test):
    #mini_test = "mini test. hello world."
    tokenizer = Tokenizer(nb_words=5)
    tokenizer.fit_on_texts(mini_test)
    text_encoded = np.zeros((len(texts),MAX_SENTENCES,MAX_SEQUENCE_LENGTH))
    #print(text_encoded.shape)
    word_index = tokenizer.word_index
    for enu, speech in enumerate(mini_test):
        sents = tokenize.sent_tokenize(speech)
        if(len(sents) == 0):
            continue
        sequences = tokenizer.texts_to_sequences(sents)
        
        #print(len(sequences))
        #print('Found %s unique tokens.' % len(word_index))
        data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
        text_encoded[enu] = (pad_dense_list(data, 100))
    return word_index, text_encoded
word_index, text_encoded = preprocess_text(texts)


In [381]:
# First Speech encoding
data = text_encoded
labels = np.asarray(labels)
print(data.shape)

(862, 100, 100)


In [382]:
new_data = data

data = data.reshape(data.shape[0], data.shape[1]*data.shape[1])


In [18]:

data = np.zeros((800, 100))
labels = np.zeros(800)

In [19]:
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)
# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
# labels = labels[indices]
VALIDATION_SPLIT = 0.2
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

Shape of data tensor: (800, 100)
Shape of label tensor: (800,)


In [20]:
embeddings_index = {}
GLOVE_DIR = '../glove.6B/'
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [8]:
EMBEDDING_DIM = 100

In [21]:
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [22]:
print(len(data))

800


In [23]:
print(data.shape)

(800, 100)


In [24]:
from keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=len(data),
                            trainable=False)

In [399]:
y_train = y_train.reshape((-1, 1))

In [402]:
print(len(x_train))

604


In [ ]:
sequence_input = Input(shape=(data.shape[1],), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = LSTM(8620)(embedded_sequences)
preds = Dense(7, activation='sigmoid')(x)
model = Model(sequence_input, preds)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

# happy learning!
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          nb_epoch=2, batch_size=1)


In [321]:
# Experiment with sequence classification 
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
np.random.seed(7)

In [329]:
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(nb_words=top_words)

In [330]:
(X_train_imdb, y_train_imdb), (X_test_imdb, y_test_imdb)  = (X_train, y_train), (X_test, y_test) 

In [339]:
print(y_train_imdb[2])

0


In [323]:
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

In [325]:
print(X_train[0])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0  

In [ ]:
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=3, batch_size=64)